In [1]:
from tensorflow.keras.models import Model

In [2]:
#/content/drive/MyDrive/AI/DL/Language_Translation/fra.txt

In [3]:
from tensorflow.keras.layers import Input, LSTM, Dense

In [4]:
import numpy as np


In [5]:
batch_size = 64

In [6]:
epochs = 100

In [7]:
latent_dim = 256

In [8]:
num_samples = 10000

In [9]:
data_path = '/content/drive/MyDrive/AI/DL/Language_Translation/fra.txt'

In [10]:
input_texts = []
target_texts = []

In [11]:
input_chars = []
target_chars = []

In [12]:
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
lines 

['Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)',
 'Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)',
 'Go.\tBouge !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)',
 'Hi.\tSalut !\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)',
 'Hi.\tSalut.\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)',
 'Run!\tCours\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906331 (sacredceltic)',
 'Run!\tCourez\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906332 (sacredceltic)',
 'Run!\tPrenez vos jambes à vos cous !\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077449 (sacredceltic)',
 'Run!\tFile !\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077454 (sacredceltic)',
 'Run!\tFilez !\tCC-BY 2.0 (France) Attribution: tatoeba

In [13]:
for line in lines[:min(num_samples,len(lines)-1)]:
  input_text, target_text, _ = line.split('\t')
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_chars:
      input_chars.append(char)
  for char in target_text:
    if char not in target_chars:
      target_chars.append(char)



In [14]:
input_chars = sorted(list(input_chars))

In [15]:
target_chars = sorted(list(target_chars))

In [16]:
num_encoder_tokens = len(input_chars)

In [17]:
num_decoder_tokens = len(target_chars)

In [18]:
max_encoder_seq_len = max([len(txt) for txt in input_texts])

In [19]:
max_decoder_seq_len = max([len(txt) for txt in target_texts])

In [20]:
len(input_texts)

10000

In [21]:
num_encoder_tokens

71

In [22]:
num_decoder_tokens

93

In [23]:
max_encoder_seq_len

15

In [24]:
max_decoder_seq_len

59

In [25]:
input_token_index = dict([(char,i) for i,char in enumerate(input_chars)])

In [26]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 'é': 70}

In [27]:
target_token_index = dict([(char,i) for i,char in enumerate(target_chars)])

In [28]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '$': 4,
 '%': 5,
 '&': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 ',': 10,
 '-': 11,
 '.': 12,
 '0': 13,
 '1': 14,
 '2': 15,
 '3': 16,
 '5': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 '?': 21,
 'A': 22,
 'B': 23,
 'C': 24,
 'D': 25,
 'E': 26,
 'F': 27,
 'G': 28,
 'H': 29,
 'I': 30,
 'J': 31,
 'K': 32,
 'L': 33,
 'M': 34,
 'N': 35,
 'O': 36,
 'P': 37,
 'Q': 38,
 'R': 39,
 'S': 40,
 'T': 41,
 'U': 42,
 'V': 43,
 'Y': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'w': 67,
 'x': 68,
 'y': 69,
 'z': 70,
 '\xa0': 71,
 '«': 72,
 '»': 73,
 'À': 74,
 'Ç': 75,
 'É': 76,
 'Ê': 77,
 'à': 78,
 'â': 79,
 'ç': 80,
 'è': 81,
 'é': 82,
 'ê': 83,
 'î': 84,
 'ï': 85,
 'ô': 86,
 'ù': 87,
 'û': 88,
 'œ': 89,
 '\u2009': 90,
 '’': 91,
 '\u202f': 92}

Total lines, Max(len(line)), number of input chareters

In [29]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_len, num_encoder_tokens),dtype='float32')

Total lines in english, max(len(line))  -French, number of french charecters

In [30]:
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')

(
  Number of lines in input data, max french line length, total number of french charecters
)


In [31]:
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')

In [32]:
target_texts

['\tVa !\n',
 '\tMarche.\n',
 '\tBouge !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tQui ?\n',
 '\tÇa alors\u202f!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tSaute.\n',
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttendez.\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttends.\n',
 '\tAttendez.\n',
 '\tCommencez.\n',
 '\tCommence.\n',
 '\tPoursuis.\n',
 '\tContinuez.\n',
 '\tPoursuivez.\n',
 '\tBonjour !\n',
 '\tSalut !\n',
 '\tJe comprends.\n',
 '\tAha.\n',
 "\tJ'essaye.\n",
 "\tJ'ai gagné !\n",
 "\tJe l'ai emporté !\n",
 '\tJ’ai gagné.\n',
 '\tOh non !\n',
 '\tCalme-toi.\n',
 '\tDétends-to

In [33]:
for i, (input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i,t,input_token_index[char]] = 1
  encoder_input_data[i,(t+1):,input_token_index[' ']] = 1  
  for t,char in enumerate(target_text):
    decoder_input_data[i,t,target_token_index[char]] = 1
    if t > 0:
      decoder_target_data[i,t-1,target_token_index[char]] = 1
  decoder_input_data[i,(t+1):,target_token_index[' ']] = 1
  decoder_target_data[i,t:,target_token_index[' ']] = 1


In [34]:
encoder_input_data.shape

(10000, 15, 71)

In [35]:
decoder_input_data.shape

(10000, 59, 93)

In [36]:
decoder_target_data.shape

(10000, 59, 93)

In [37]:
encoder_inputs = Input(shape=(None,num_encoder_tokens))

In [38]:
encoder = LSTM(latent_dim,return_state=True)

In [39]:
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [40]:
encoder_states = [state_h, state_c]

In [41]:
decoder_inputs = Input(shape=(None,num_decoder_tokens))

In [42]:
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_lstm

In [43]:
decoder_inputs

<KerasTensor: shape=(None, None, 93) dtype=float32 (created by layer 'input_2')>

In [44]:
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)

In [45]:
decoder_dense = Dense(num_decoder_tokens,activation='softmax')

In [46]:
decoder_outputs = decoder_dense(decoder_outputs)

In [47]:
model = Model([encoder_inputs,decoder_inputs], decoder_outputs)

In [48]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [49]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_data, batch_size=batch_size,epochs=epochs,validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 34s 21ms/step - loss: 1.5068 - accuracy: 0.7035 - val_loss: 1.1131 - val_accuracy: 0.7103
Epoch 2/100
125/125 [==============================] - 2s 14ms/step - loss: 0.8854 - accuracy: 0.7631 - val_loss: 0.8340 - val_accuracy: 0.7651
Epoch 3/100
125/125 [==============================] - 2s 14ms/step - loss: 0.7068 - accuracy: 0.8051 - val_loss: 0.7098 - val_accuracy: 0.7949
Epoch 4/100
125/125 [==============================] - 2s 14ms/step - loss: 0.6006 - accuracy: 0.8268 - val_loss: 0.6445 - val_accuracy: 0.8125
Epoch 5/100
125/125 [==============================] - 2s 14ms/step - loss: 0.5418 - accuracy: 0.8416 - val_loss: 0.6012 - val_accuracy: 0.8256
Epoch 6/100
125/125 [==============================] - 2s 14ms/step - loss: 0.5038 - accuracy: 0.8524 - val_loss: 0.5679 - val_accuracy: 0.8335
Epoch 7/100
125/125 [==============================] - 2s 14ms/step - loss: 0.4735 - accuracy: 0.8605 - val_loss: 0.5458 - val_accuracy

In [50]:
encoder_model = Model(encoder_inputs,encoder_states)
# decoder_statr

In [51]:
decoder_state_input_h = Input(shape=(latent_dim,))

In [52]:
decoder_state_input_c = Input(shape=(latent_dim,))

In [53]:
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [54]:
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs,initial_state=decoder_states_inputs)

In [55]:
decoder_states = [state_h, state_c]

In [56]:
decoder_outputs = decoder_dense(decoder_outputs)

In [57]:
decoder_model = Model([decoder_inputs]+decoder_states_inputs, [decoder_outputs]+decoder_states)

In [58]:
def decode_sequence(input_seq):
  states_value = encoder_model.predict(input_seq)
  target_seq = np.zeros((1,1,num_decoder_tokens))
  target_seq[0,0,target_token_index['\t']] = 1
  stop_condition = False
  decoded_sentense = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq]+states_value)
    sample_token_index = np.argmax(output_tokens[0,-1,:])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentense +=sampled_char
    if sampled_char == '\n'or len(decoded_sentense) > max_decoder_seq_len:
      stop_condition = True 
    target_seq = np.zeros((1,1,num_decoder_tokens))
    target_seq[0,0,sampled_token_index] = 1
    states_value = [h,c]
  return decoded_sentense

In [59]:
# from tensorflow.keras.layers import Lambda

In [60]:
# from tensorflow.keras import backend as K

In [61]:
# encoder_inputs = Input(shape=(None,num_encoder_tokens))

In [62]:
# encoder = LSTM(latent_dim,return_state=True)

In [63]:
# encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [64]:
# states = [state_h, state_c]

In [65]:
# decoder_inputs = Input(shape=(1,num_decoder_tokens))

In [66]:
# decoder_lstm = LSTM(latent_dim,return_sequences=True,return_state=True)

In [67]:
# decoder_dense = Dense(num_decoder_tokens,activation='softmax')

In [68]:
# inputs = decoder_inputs

In [69]:
# inputs

In [70]:
# all_outputs = []
# for _ in range(max_decoder_seq_len):
#   outputs, state_h, state_c = decoder_lstm(inputs, initial_state=states)
#   outputs = decoder_dense(outputs)
#   all_outputs.append(outputs)
#   inputs = outputs
#   states = [state_h, state_c]
# decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)
# model = Model([encoder_inputs,decoder_inputs], decoder_outputs)
# model.compile(optimizer='rmsprop',loss='categorical_crossentropy')
# decoder_input_data = np.zeros((num_samples,1,num_decoder_tokens))
# decoder_input_data[:,0,target_token_index['\t']] = 1
